In [2]:
# Adaptado para series temporales: TimeSeriesSplit + Early Stopping

import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import joblib

# Cargar datos
path = '/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/trading/project_trading_DT/data/processed/TLT_clean.csv'
df = pd.read_csv(path)

# Separar entrenamiento y prueba
train_df = df[df['Date'] <= '2025-02-28']
test_df = df[df['Date'] >= '2025-03-01']

selected_features = [  # Copiar de tu script de generación de features
    "day_of_week", "is_month_end", "month",
    "price_diff", "pct_diff", "log_vol",
    "return_lag_1", "return_lag_2", "return_lag_3", "return_lag_4", "return_lag_5",
    "sma_5", "ema_5", "rolling_std_return_5",
    "rsi_5", "macd", "macd_signal", "macd_above_signal",
    "return_volatility_ratio",
    "us10y_yield", "fed_rate", "cpi", "unemployment_rate"
]

X_train = train_df[selected_features].copy()
y_train = train_df['target']
X_test = test_df[selected_features].copy()
y_test = test_df['target']

# Validación para early stopping (10% final del entrenamiento)
val_size = int(len(X_train) * 0.1)
X_train_base, X_val = X_train.iloc[:-val_size], X_train.iloc[-val_size:]
y_train_base, y_val = y_train.iloc[:-val_size], y_train.iloc[-val_size:]

# Funciones

from xgboost.callback import EarlyStopping

from lightgbm import early_stopping


def train_and_evaluate(model, X_train, X_test, y_train, y_test, model_name, X_val=None, y_val=None):
    if isinstance(model, XGBClassifier) and X_val is not None:
        model.set_params(early_stopping_rounds=10, eval_metric="error")
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            verbose=False
        )
    elif isinstance(model, lgb.LGBMClassifier) and X_val is not None:
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="error",
            callbacks=[early_stopping(10)]
        )
    elif isinstance(model, CatBoostClassifier) and X_val is not None:
        model.fit(
            X_train,
            y_train,
            eval_set=(X_val, y_val),
            early_stopping_rounds=10
        )
    else:
        model.fit(X_train, y_train)

    # Predecir probabilidades
    y_proba = model.predict_proba(X_test)[:, 1]

    # Buscar mejor threshold entre 0.3 y 0.7
    best_thresh = 0.5
    best_macro_prec = 0
    for thresh in np.arange(0.3, 0.71, 0.05):
        y_pred_temp = (y_proba > thresh).astype(int)
        macro_prec = precision_score(y_test, y_pred_temp, average="macro", zero_division=0)
        if macro_prec > best_macro_prec:
            best_macro_prec = macro_prec
            best_thresh = thresh

    print(f"✅ Mejor threshold para {model_name}: {best_thresh:.2f} con precisión macro: {best_macro_prec:.4f}")

    # Usar el mejor threshold para predicción final
    y_pred = (y_proba > best_thresh).astype(int)

    print(f'{model_name} Accuracy: {accuracy_score(y_test, y_pred):.4f}')
    print(f'{model_name} Precision (Macro): {precision_score(y_test, y_pred, average="macro"):.4f}')
    print(f'{model_name} Recall (Macro): {recall_score(y_test, y_pred, average="macro"):.4f}')
    print(f'Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}')
    print(f'Classification Report:\n{classification_report(y_test, y_pred)}')

    if hasattr(model, 'feature_importances_'):
        print("Feature Importances:")
        for name, importance in zip(X_train.columns, model.feature_importances_):
            print(f"{name}: {importance:.4f}")

    model.best_threshold = best_thresh

    return model


from sklearn.metrics import make_scorer, precision_score

# Métrica personalizada para solo la clase positiva (label 1)
positive_precision = make_scorer(precision_score, pos_label=1, average='binary', zero_division=0)

def hyperparameter_tuning(model, X, y, param_grid):
    tscv = TimeSeriesSplit(n_splits=10)
    search = RandomizedSearchCV(
        model,
        param_distributions=param_grid,
        n_iter=20,
        cv=tscv,
        scoring=positive_precision,  # Aquí cambiamos la métrica
        n_jobs=-1,
        random_state=42
    )
    search.fit(X, y)
    print(f"Best params for {model.__class__.__name__}: {search.best_params_}")
    return search.best_estimator_


# Modelos y grids
models = {
    "XGBoost": XGBClassifier(random_state=42),
    "LightGBM": lgb.LGBMClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(random_state=42, verbose=0),
    "RandomForest": RandomForestClassifier(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42)
}

param_grids = {
    "XGBoost": {
        "n_estimators": [100, 300, 500],
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 5, 10]
    },
    "LightGBM": {
        "n_estimators": [100, 300, 500],
        "learning_rate": [0.01, 0.1],
        "num_leaves": [31, 50, 70]
    },
    "CatBoost": {
        "iterations": [500, 1000],
        "learning_rate": [0.01, 0.1],
        "depth": [6, 10]
    },
    "RandomForest": {
        "n_estimators": [100, 300],
        "max_depth": [10, 20],
        "min_samples_split": [2, 5]
    },
    "DecisionTree": {
        "max_depth": [3, 5, 10],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 5]
    },
    "GradientBoosting": {
        "n_estimators": [100, 200],
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 5]
    },
    "AdaBoost": {
        "n_estimators": [50, 100],
        "learning_rate": [0.01, 0.1]
    }
}

# Entrenamiento
best_model = None
best_score = -np.inf

trained_models = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    tuned_model = hyperparameter_tuning(model, X_train_base, y_train_base, param_grids[name])
    final_model = train_and_evaluate(tuned_model, X_train_base, X_test, y_train_base, y_test, name, X_val=X_val, y_val=y_val)
    
    trained_models[name] = final_model  # ✅ guardar el modelo entrenado

    # Evaluar precisión y guardar mejor modelo
    score = precision_score(y_test, final_model.predict(X_test), average='macro')
    if score > best_score:
        best_score = score
        best_model = final_model

print(f"\nBest Model: {best_model} with macro precision {best_score:.4f}")


Training XGBoost...


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 18 is smaller than n_iter=20. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best params for XGBClassifier: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.1}
✅ Mejor threshold para XGBoost: 0.60 con precisión macro: 0.8269
XGBoost Accuracy: 0.7000
XGBoost Precision (Macro): 0.8269
XGBoost Recall (Macro): 0.6538
Confusion Matrix:
[[17  0]
 [ 9  4]]
Classification Report:
              precision    recall  f1-score   support

           0       0.65      1.00      0.79        17
           1       1.00      0.31      0.47        13

    accuracy                           0.70        30
   macro avg       0.83      0.65      0.63        30
weighted avg       0.80      0.70      0.65        30

Feature Importances:
day_of_week: 0.0414
is_month_end: 0.0702
month: 0.0352
price_diff: 0.0418
pct_diff: 0.0441
log_vol: 0.0392
return_lag_1: 0.0402
return_lag_2: 0.0385
return_lag_3: 0.0406
return_lag_4: 0.0389
return_lag_5: 0.0423
sma_5: 0.0373
ema_5: 0.0569
rolling_std_return_5: 0.0417
rsi_5: 0.0416
macd: 0.0426
macd_signal: 0.0433
macd_above_signal: 0.0373
ret

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 18 is smaller than n_iter=20. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[LightGBM] [Info] Number of positive: 157, number of negative: 133
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002348 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1514
[LightGBM] [Info] Number of data points in the train set: 290, number of used features: 22
[LightGBM] [Info] Number of positive: 1383, number of negative: 1219
[LightGBM] [Info] Number of positive: 938, number of negative: 797
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.541379 -> initscore=0.165897
[LightGBM] [Info] Start training from score 0.165897
[LightGBM] [Info] Number of positive: 784, number of negative: 662
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004085 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was

...

Best params for GradientBoostingClassifier: {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1}
✅ Mejor threshold para GradientBoosting: 0.70 con precisión macro: 0.6827
GradientBoosting Accuracy: 0.6333
GradientBoosting Precision (Macro): 0.6827
GradientBoosting Recall (Macro): 0.5860
Confusion Matrix:
[[16  1]
 [10  3]]
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.94      0.74        17
           1       0.75      0.23      0.35        13

    accuracy                           0.63        30
   macro avg       0.68      0.59      0.55        30
weighted avg       0.67      0.63      0.57        30

Feature Importances:
day_of_week: 0.0204
is_month_end: 0.0027
month: 0.0079
price_diff: 0.0789
pct_diff: 0.0498
log_vol: 0.0831
return_lag_1: 0.0789
return_lag_2: 0.0666
return_lag_3: 0.0861
return_lag_4: 0.0447
return_lag_5: 0.0640
sma_5: 0.0374
ema_5: 0.0535
rolling_std_return_5: 0.0531
rsi_5: 0.0741
macd: 0.0506
m

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/lib

Best params for AdaBoostClassifier: {'n_estimators': 100, 'learning_rate': 0.1}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


✅ Mejor threshold para AdaBoost: 0.50 con precisión macro: 0.4861
AdaBoost Accuracy: 0.5000
AdaBoost Precision (Macro): 0.4861
AdaBoost Recall (Macro): 0.4864
Confusion Matrix:
[[10  7]
 [ 8  5]]
Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.59      0.57        17
           1       0.42      0.38      0.40        13

    accuracy                           0.50        30
   macro avg       0.49      0.49      0.49        30
weighted avg       0.50      0.50      0.50        30

Feature Importances:
day_of_week: 0.0500
is_month_end: 0.0000
month: 0.0000
price_diff: 0.1200
pct_diff: 0.0000
log_vol: 0.1700
return_lag_1: 0.0900
return_lag_2: 0.0200
return_lag_3: 0.1200
return_lag_4: 0.0200
return_lag_5: 0.0100
sma_5: 0.0500
ema_5: 0.0600
rolling_std_return_5: 0.0200
rsi_5: 0.0700
macd: 0.0200
macd_signal: 0.0800
macd_above_signal: 0.0000
return_volatility_ratio: 0.0600
us10y_yield: 0.0000
fed_rate: 0.0000
cpi: 0.0400
unemployme

In [5]:
from sklearn.metrics import precision_score

# Agregar columna Close_next para calcular retornos reales
test_df["Close_next"] = test_df["Close"].shift(-1)
test_df.dropna(inplace=True)
X_test_sim = test_df[selected_features].reset_index(drop=True)
y_test_sim = test_df["target"].reset_index(drop=True)

# 🔧 Capital inicial y fracción de inversión por operación
initial_capital = 1000
investment_fraction = 0.25
investment_per_trade = initial_capital * investment_fraction  # $250 por trade

results = []
thresholds = [0.5, 0.6, 0.7]

for name, model in trained_models.items():
    y_proba = model.predict_proba(X_test_sim)[:, 1]

    for threshold in thresholds:
        print(f"\n💰 Simulando inversión con modelo: {name} | Threshold: {threshold}")
        
        y_pred = (y_proba > threshold).astype(int)

        temp_df = test_df.copy()
        temp_df["pred"] = y_pred

        # 🔧 Calcular retorno sobre la inversión de $250 por operación
        temp_df["retorno_diario"] = temp_df["Close_next"] / temp_df["Close"] - 1
        temp_df["ganancia"] = np.where(temp_df["pred"] == 1, investment_per_trade * temp_df["retorno_diario"], 0)

        total_gain = temp_df["ganancia"].sum()
        final_capital = initial_capital + total_gain
        trades = temp_df["pred"].sum()

        results.append({
            "Modelo": name,
            "Threshold": threshold,
            "Trades realizados": int(trades),
            "Ganancia Total ($)": round(total_gain, 2),
            "Capital Final ($)": round(final_capital, 2),
            "Precision (1)": round(precision_score(y_test_sim, y_pred, zero_division=0), 3)
        })

# 🔧 Mostrar resultados ordenados como DataFrame
import pandas as pd
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="Capital Final ($)", ascending=False)

print("\n🏁 Resultados de la simulación con múltiples thresholds:")
print(results_df.to_string(index=False))


💰 Simulando inversión con modelo: XGBoost | Threshold: 0.5

💰 Simulando inversión con modelo: XGBoost | Threshold: 0.6

💰 Simulando inversión con modelo: XGBoost | Threshold: 0.7

💰 Simulando inversión con modelo: LightGBM | Threshold: 0.5

💰 Simulando inversión con modelo: LightGBM | Threshold: 0.6

💰 Simulando inversión con modelo: LightGBM | Threshold: 0.7

💰 Simulando inversión con modelo: CatBoost | Threshold: 0.5

💰 Simulando inversión con modelo: CatBoost | Threshold: 0.6

💰 Simulando inversión con modelo: CatBoost | Threshold: 0.7

💰 Simulando inversión con modelo: RandomForest | Threshold: 0.5

💰 Simulando inversión con modelo: RandomForest | Threshold: 0.6

💰 Simulando inversión con modelo: RandomForest | Threshold: 0.7

💰 Simulando inversión con modelo: DecisionTree | Threshold: 0.5

💰 Simulando inversión con modelo: DecisionTree | Threshold: 0.6

💰 Simulando inversión con modelo: DecisionTree | Threshold: 0.7

💰 Simulando inversión con modelo: GradientBoosting | Threshold:

/var/folders/w6/tt5g659n54zdxvlqlfd5dy3c0000gn/T/ipykernel_56489/811258384.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["Close_next"] = test_df["Close"].shift(-1)
/var/folders/w6/tt5g659n54zdxvlqlfd5dy3c0000gn/T/ipykernel_56489/811258384.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.dropna(inplace=True)
